In [1]:
import findspark
findspark.init('/home/osboxes/spark-2.4.3-bin-hadoop2.7')
import pyspark
import os

In [2]:
myPath = os.path.join('/home', 'osboxes', 'CourseMaterial', 'Spark_for_Machine_Learning', 'Natural_Language_Processing')

print(myPath)

os.chdir(myPath)

/home/osboxes/CourseMaterial/Spark_for_Machine_Learning/Natural_Language_Processing


In [3]:
os.listdir()

['Tools_for_NLP.ipynb',
 'NLP_Code_Along.ipynb',
 '.ipynb_checkpoints',
 'smsspamcollection']

In [4]:
# set up the session info

from pyspark.sql import SparkSession

name = 'nlp'

spark = SparkSession.builder.appName(name).getOrCreate()

# nlp tools
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [5]:
# default splitter for tokenizer is space
# create dataframe as list of tuples

sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

In [6]:
sentenceDataFrame.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [7]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [8]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words',
                                pattern='\\W')

In [9]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [10]:
# id 2 used commas and not spaces, check split

tokenized = tokenizer.transform(sentenceDataFrame)
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [11]:
# did not split the last sentence since no spaces

tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [12]:
rg_tokenized = regex_tokenizer.transform(sentenceDataFrame)
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [13]:
# stopwords remover

from pyspark.ml.feature import StopWordsRemover

In [14]:
sentenceData = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "raw"])

In [15]:
sentenceData.show()

+---+--------------------+
| id|                 raw|
+---+--------------------+
|  0|[I, saw, the, red...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [16]:
remover = StopWordsRemover(inputCol='raw', outputCol='filtered')
remover.transform(sentenceData).show()

+---+--------------------+--------------------+
| id|                 raw|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, red...| [saw, red, balloon]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [17]:
# n-grams

from pyspark.ml.feature import NGram

In [18]:
wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

wordDataFrame.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|[Hi, I, heard, ab...|
|  1|[I, wish, Java, c...|
|  2|[Logistic, regres...|
+---+--------------------+



In [21]:
ngram = NGram(n=2, inputCol='words', outputCol='grams')
ngram.transform(wordDataFrame).select('grams').show(truncate=False)

+------------------------------------------------------------------+
|grams                                                             |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [22]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [23]:
sentenceDataFrame.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [24]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')
words_data = tokenizer.transform(sentenceDataFrame)
words_data.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [29]:
# build up tf-idf functions

hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')
featurized_data = hashing_tf.transform(words_data)
idf = IDF(inputCol='rawFeatures', outputCol='features')
idf_model = idf.fit(featurized_data)

In [31]:
rescaled_data = idf_model.transform(featurized_data)


In [34]:
rescaled_data.select(['id', 'features']).show()

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|(262144,[24417,49...|
|  1|(262144,[20719,24...|
|  2|(262144,[84234],[...|
+---+--------------------+



In [35]:
rescaled_data.select(['id', 'features']).show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                                                                                        |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0  |(262144,[24417,49304,73197,91137,234657],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                     |
|1  |(262144,[20719,24417,55551,116873,147765,162369,192310],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|2  |

In [36]:
# count vectorizers

from pyspark.ml.feature import CountVectorizer

In [37]:
# Input data: Each row is a bag of words with a ID.
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [38]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [39]:
# fit a CountVectorizerModel from the corpus
# min df cuts off minumum count of documents for a word to show up

cv = CountVectorizer(inputCol="words", outputCol="features", vocabSize=3, minDF=2.0)

In [40]:
model = cv.fit(df)

In [41]:
result = model.transform(df)
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

